In [2]:
# pip install langchain-qdrant

In [3]:
from langchain_community.embeddings import OpenAIEmbeddings
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant

In [11]:
import os
from dotenv import load_dotenv

# Specify the path to your .env file

ENV_PATH = '/mnt/c/Users/user/OneDrive/Desktop/llm-apps/mental-disorder-ai-agent/.env'

# Load environment variables from the specified .env file
load_dotenv(dotenv_path=ENV_PATH)

True

In [6]:
from langchain.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("/mnt/c/Users/user/OneDrive/Desktop/llm-apps/mental-disorder-ai-agent/data")
data= loader.load_and_split()

In [7]:
?? data

Type:        list
String form: [Document(metadata={'source': '/mnt/c/Users/user/OneDrive/Desktop/llm-apps/mental-disorder-ai-agent/data/generalized_anxiety_disorder.pdf', 'page': 0}, page_content='Generalized \nAnxiety Disorder:\nWhen Worry Gets \nOut of Control'), Document(metadata={'source': '/mnt/c/Users/user/OneDrive/Desktop/llm-apps/mental-disorder-ai-agent/data/generalized_anxiety_disorder.pdf', 'page': 1}, page_content='Do you often find yourself worrying about everyday \nissues for no obvious reason? Are you always waiting for disaster to strike or excessively worried about things such as health, money, family, work, or school?\nIf so, you may have a type of anxiety disorder called \ngeneralized anxiety disorder (GAD). GAD can make daily life feel like a constant state of worry, fear, and dread. The good news is GAD is treatable. Learn more about the symptoms of GAD and how to find help.\nWhat is generalized anxiety disorder?\nOccasional anxiety is a normal part of life. Many p

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(data)


In [10]:
?? documents

Type:        list
String form: [Document(metadata={'source': '/mnt/c/Users/user/OneDrive/Desktop/llm-apps/mental-disorder-ai-agent/data/generalized_anxiety_disorder.pdf', 'page': 0}, page_content='Generalized \nAnxiety Disorder:\nWhen Worry Gets \nOut of Control'), Document(metadata={'source': '/mnt/c/Users/user/OneDrive/Desktop/llm-apps/mental-disorder-ai-agent/data/generalized_anxiety_disorder.pdf', 'page': 1}, page_content='Do you often find yourself worrying about everyday \nissues for no obvious reason? Are you always waiting for disaster to strike or excessively worried about things such as health, money, family, work, or school?\nIf so, you may have a type of anxiety disorder called \ngeneralized anxiety disorder (GAD). GAD can make daily life feel like a constant state of worry, fear, and dread. The good news is GAD is treatable. Learn more about the symptoms of GAD and how to find help.\nWhat is generalized anxiety disorder?\nOccasional anxiety is a normal part of life. Many p

In [12]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
def create_db(documents):
    return Qdrant.from_documents(
        documents=documents,
        embedding=embeddings,
        collection_name="my_documents",
        location=":memory:",
        force_recreate=False,
    )
db = create_db(documents)

/home/sina/miniconda3/envs/langchain_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Build an autonomous Agents: everything-about-book

#### define Tools

In [13]:
from langchain.tools import BaseTool, StructuredTool, tool

In [16]:
# pip install langchain-openai

In [18]:
from langchain_openai import ChatOpenAI

from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)

In [19]:
session_state = {}

In [20]:
@tool
def find_relevant_books(user_query):
    """
    Return all relevant books based on user query.
    Important: This function should be called only for queries that require finding specific books.
    For general queries that do not require finding specific books, use other available functions.
    """
    retriever = db.as_retriever(
        search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.25}
    )
    relevant_docs = retriever.get_relevant_documents(user_query)
    session_state["relevant_docs"] = relevant_docs
    session_state["retriever"] = retriever
    return relevant_docs

llm = ChatOpenAI(
    model="gpt-4o", 
    temperature=0, 
    openai_api_key=os.getenv("OPEN_AI_KEY")
)
@tool
def qa(user_query):
    """
    Answer user questions based on the retrieved documents
    """
    retriever = session_state["retriever"]
    relevant_docs = session_state.get("relevant_docs")
    if relevant_docs is None:
        # If no documents are stored, retrieve them
        relevant_docs = retriever.get_relevant_documents(user_query)
        session_state["relevant_docs"] = relevant_docs
    
    # Create a chain to answer questions using stored documents
    qa = ConversationalRetrievalChain.from_llm(llm, retriever)
    chat_history = []
    result = qa(
        {"question": user_query, "chat_history": chat_history, "context": relevant_docs}
    )
    return result

In [21]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define the prompt template
prompt_template = """
You are a helpful AI assistant specializing in answering questions 
related to books from users. Use retrieved relevant books to 
answer questions.
====================
{relevant_docs}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are helpful AI assistant. Use the following 
               template for your actions and observations."""
        ),
        ("user", prompt_template),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [27]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.tools import format_tool_to_openai_function
from langchain.tools import Tool

In [29]:
# These are custom functions for finding books, answering questions, and creating topic networks.
tools = [find_relevant_books, qa]
# OpenAI Function Formatting. This converts the tools into a format compatible with OpenAI's function calling feature.
functions = [format_tool_to_openai_function(f) for f in tools]
#This sets up the GPT-4o model with the defined functions.
model = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o",
).bind(functions=functions)

In [30]:
from langchain.agents import AgentExecutor
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

In [31]:
# Set up the agent chain.
# including assigning relevant documents and agent scratchpad, applying the prompt, running the model, and parsing the output.
agent_chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"]),
        relevant_docs=lambda x: "\n".join(
            str(doc) for doc in session_state.get("relevant_docs", [])
        ),
    )
    | prompt
    | model
    | OpenAIFunctionsAgentOutputParser()
)
# Set up a memory component to store conversation history.
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history",
    input_key="input",
    output_key="output",
)
# Initialize an agent with the agent and defined tools
# This combines all components into an executable agent that can process queries and maintain conversation context.
# With AgentExecutor, the agent is equipped with the tools and verbose output is enabled, allowing for detailed logging.
agent = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [32]:
agent.invoke({"input": "what are symptom of PTSD?"})



> Entering new AgentExecutor chain...

Invoking: `find_relevant_books` with `{'user_query': 'symptoms of PTSD'}`




/home/sina/miniconda3/envs/langchain_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'source': '/mnt/c/Users/user/OneDrive/Desktop/llm-apps/mental-disorder-ai-agent/data/ptsd_open_source.pdf', 'page': 2, '_id': 'aa33f2577943470c98ac3a850b224e52', '_collection_name': 'my_documents'}, page_content='To be diagnosed with PTSD, an adult must have all of the following for  \nat least 1 month:\n ⊲At least one re-experiencing symptom\n ⊲At least one avoidance symptom\n ⊲At least two arousal and reactivity symptoms\n ⊲At least two cognition and mood symptoms\nRe-experiencing symptoms\n⊲ Flashbacks—reliving the traumatic event, including physical symptoms such as a racing heart or sweating\n⊲ Reoccurring memories or dreams related to the event\n⊲ Distressing thoughts\n⊲ Physical signs of stress\nThoughts and feelings can trigger these symptoms, as can words, objects, or situations that are reminders of the event.\nAvoidance symptoms\n⊲ Staying away from places, events, or objects that are reminders of the experience\n⊲ Avoiding thoughts or feelings related to

{'input': 'what are symptom of PTSD?',
 'chat_history': [HumanMessage(content='what are symptom of PTSD?'),
  AIMessage(content='To be diagnosed with PTSD (Post-Traumatic Stress Disorder), an adult must have the following symptoms for at least one month:\n\n1. **Re-experiencing Symptoms**:\n   - Flashbacks: Reliving the traumatic event, including physical symptoms such as a racing heart or sweating.\n   - Reoccurring memories or dreams related to the event.\n   - Distressing thoughts.\n   - Physical signs of stress.\n\n2. **Avoidance Symptoms**:\n   - Staying away from places, events, or objects that are reminders of the experience.\n   - Avoiding thoughts or feelings related to the traumatic event.\n   - These symptoms may cause people to change their routines, such as avoiding driving or riding in a car after a serious car accident.\n\n3. **Arousal and Reactivity Symptoms**:\n   - Being easily startled.\n   - Having trouble falling asleep or staying asleep.\n\n4. **Cognition and Mood